## 从雅虎财经网站获取股票数据

In [20]:
import urllib
import os
import datetime
import pandas as pd


In [21]:
# 下载整个股票数据
def retrive_stock_data(stockid, folder):
    print('downloading %s to %s' % (stockid, folder))
    url = 'http://table.finance.yahoo.com/table.csv?s=%s' % (stockid)
    fname = os.path.join(folder, '%s.csv' % stockid.split('.')[0])
    if not os.path.isdir(folder):
        os.mkdir(folder)
    urllib.urlretrieve(url, fname)

In [24]:
retrive_stock_data('600690.ss', 'yahoo-data')

downloading 600690.ss to yahoo-data


In [35]:
# 更新股票数据，如果不存在，则下载。如果存在，则只更新最近日期的数据
def update_stock_data(stockid, folder):
    fname = os.path.join(folder, '%s.csv' % stockid.split('.')[0])
    if not os.path.exists(fname):
        retrive_stock_data(stockid, folder)
        return
    
    data = pd.read_csv(fname, index_col='Date', parse_dates=True)
    
    last_date = data.iloc[0:1].index.tolist()[0]
    today = pd.Timestamp(datetime.date.today())
    if today - last_date < pd.Timedelta(days=2):
        print('Nothing to update. %s last date is %s.' % (stockid, last_date))
        return
    
    print('updatting %s to from %s to %s' % (stockid, last_date.date(), today.date()))
    query = [
        ('a', last_date.month - 1),
        ('b', last_date.day),
        ('c', last_date.year),
        ('d', today.month - 1),
        ('e', today.day),
        ('f', today.year),
        ('s', stockid),
    ]
    url = 'http://table.finance.yahoo.com/table.csv?%s' % urllib.urlencode(query)
    temp_file = fname + '.tmp'
    urllib.urlretrieve(url, temp_file)
    update_data = pd.read_csv(temp_file, index_col='Date', parse_dates=True)
    data = data.append(update_data)
    data.sort_index(ascending=False, inplace=True)
    data.to_csv(fname, mode='w')
    os.unlink(temp_file)

In [36]:
update_stock_data('600690.ss', 'yahoo-data')

updatting 600690.ss to from 2015-10-27 to 2015-11-26


In [19]:
query = [('a', 10), ('b', 5), ('c', 2015), ('d', 10), ('e', 25), ('f', 2015), ('s', '600690.ss')]
urllib.urlencode(query)

'a=10&b=5&c=2015&d=10&e=25&f=2015&s=600690.ss'